## Interacting with CSV Data

#### Setup

In [ ]:
import os 
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4-1106",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

#### Load the dataset

In [ ]:
# Load the dataset
df = pd.read_csv(r"./data/all_data.csv").fillna(value=0)

#### Prepare the Langchain dataframe agent

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model, df=df, verbose=True)

In [ ]:
agent.invoke("how many rows are there?")

#### Design the prompt and ask questions

In [ ]:
CSV_PROMPT_PREFIX = """
first set the pandas display options to show all columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the final answer, try another method.
Then reflect on the answers of the two methods and ask yourself whether it 
answers the original question correctly.
If you are not sure, try another method.
- If the methods tried do not give the same result, reflect and try again
until you have two methods that give the same result.
- If you still cannot arrive at a confident answer, state that you are unsure
and explain why.
- If you are sure of the correct answer, create a beautiful and thorough response
using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF
THE METHODS TRIED TO ARRIVE AT THE FINAL ANSWER.**
- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer
on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column anmes that you used to get to the final answer.
"""

In [ ]:
QUESTION = "How many patients were hospitalised during July 2020 " 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column to determine this."

agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)